In [2]:
import pandas as pd
import json
from scipy import stats
import numpy as np

# 이벤트 기간에 가입한 유저

### 가입 전체 유저

In [4]:
read_path = '../csv/reward_register_2.0_all_user.csv'
all_user = pd.read_csv(read_path)

all_user['uid'] = all_user['uid'].astype(str)

all_user.tail()

,join_date,uid
16678,2019-12-08,10410217
16679,2019-12-08,10410218
16680,2019-12-08,10410219
16681,2019-12-08,10410220
16682,2019-12-08,10410221


### 이벤트 대상 유저

In [ ]:
read_path = '../csv/ㅇㄹ.csv'
target_user = pd.read_csv(read_path)

target_user = target_user[target_user['전송 실패'] != '전송 실패'] # 번개톡 전송 실패 제외

target_user = target_user.rename(columns={'date': 'event_date'})

target_user = target_user[['event_date', 'uid', 'test_group']]

target_user['uid'] = target_user['uid'].astype(str)

target_user.tail()

### 전체 가입 유저와 이벤트 대상 유저 merge